In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import thinkstats2
import thinkplot
from sklearn.feature_selection import VarianceThreshold
plt.style.use('fivethirtyeight')
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from scipy.stats import norm
from scipy import stats

In [ ]:
train = pd.read_csv('G:/Desktop/datasets/flight-delays-spring-2018/flight_delays_train.csv')
test = pd.read_csv('G:/Desktop/datasets/flight-delays-spring-2018/flight_delays_test.csv')
train.head()

In [ ]:
data = train.copy()

In [ ]:
data.columns

In [ ]:
data.dtypes

# dep_delayed_15min

In [ ]:
data['dep_delayed_15min'].describe()

In [ ]:
print(data['dep_delayed_15min'].value_counts(normalize=True))
sns.countplot(data['dep_delayed_15min'])

In [ ]:
data['dep_delayed_15min'].unique()

In [ ]:
data['dep_delayed_15min'] = data['dep_delayed_15min'].map({'N':0, 'Y':1})

# Distance

In [ ]:
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='dep_delayed_15min', y='Distance', data=data)

# month

In [ ]:
d = {'c-8':8, 'c-4':4, 'c-9':9, 'c-11':11, 'c-10':10, 'c-1':1, 'c-7':7, 'c-6':6, 'c-5':5,
       'c-3':3, 'c-12':12, 'c-2':2}
data['Month'] = data['Month'].map(d)

In [ ]:
yes_pmf = thinkstats2.Pmf(data[data['dep_delayed_15min'] == 1]['Month'], label='1')
no_pmf = thinkstats2.Pmf(data[data['dep_delayed_15min'] == 0]['Month'],  label='0')
plt.figure(figsize=(10, 7))
thinkplot.Pmfs([yes_pmf, no_pmf])
thinkplot.Show(xlabel='Month', ylabel='PMF')

# destination

In [ ]:
le = LabelEncoder()

In [ ]:
le.fit(data['Dest'])    

In [ ]:
a = le.transform(data['Dest'])

In [ ]:
data['Dest'] = pd.Series(a)

In [ ]:
yes_pmf = thinkstats2.Cdf(data[data['dep_delayed_15min'] == 1]['Dest'], label='Yes')
no_pmf = thinkstats2.Cdf(data[data['dep_delayed_15min'] == 0]['Dest'],  label='No')
plt.figure(figsize=(12, 8))
thinkplot.Cdfs([yes_pmf, no_pmf])
thinkplot.Show(xlabel='Dest', ylabel='Cdf')

In [ ]:
plt.figure(figsize=(12, 7))
sns.pairplot(data)

In [ ]:
data.dtypes

In [ ]:
plt.figure(figsize=(12, 8))
sns.distplot(data['Distance'])

In [ ]:
data['Distance'] = np.log(data['Distance'])

In [ ]:
plt.figure(figsize=(12, 8))
sns.distplot(data['Distance'], fit=norm);
fig = plt.figure(figsize=(12, 8))
res = stats.probplot(data['Distance'], plot=plt)

In [ ]:
data['Distance'].describe()

In [ ]:
indexes = data[data['Distance'] < 4.3].index

In [ ]:
data = data.drop(data.index[indexes], axis=0)

In [ ]:
plt.figure(figsize=(12, 8))
sns.distplot(data['Distance'], fit=norm);
fig = plt.figure(figsize=(12, 8))
res = stats.probplot(data['Distance'], plot=plt)

In [ ]:
data = pd.get_dummies(data)

In [ ]:
X = data.drop('dep_delayed_15min', axis=1)
y = data['dep_delayed_15min']

In [ ]:
X = StandardScaler().fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

In [ ]:
lr.score(X_test, y_test)

In [ ]:
%%time
extra = ExtraTreesClassifier()
extra = extra.fit(X_train,y_train)

In [ ]:
extra.score(X_test, y_test)

In [ ]:
%%time
ensemble_lin_rbf=VotingClassifier(estimators=[('RBF',SVC(probability=True,kernel='rbf',C=0.5,gamma=0.1)),
                                              ('RFor',RandomForestClassifier(n_estimators=500,random_state=0)),
                                              ('LR',LogisticRegression(C=0.05)),
                                              ('xgb',XGBClassifier()),
                                              ('svm',SVC(kernel='linear',probability=True)),
                                              ('ada',AdaBoostClassifier(n_estimators=200,random_state=0,learning_rate=0.1)),
                                              ('gbc',GradientBoostingClassifier(n_estimators=500,random_state=0,learning_rate=0.1))
                                             ], 
                       voting='soft').fit(X_train[:40000],y_train[:40000])

In [ ]:
ensemble_lin_rbf.score(X_test, y_test)

In [ ]:
%%time
ada=AdaBoostClassifier(n_estimators=1000,random_state=0,learning_rate=0.1)
ada.fit(X_train, y_train)

In [ ]:
ada.score(X_test, y_test)

In [ ]:
%%time
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

In [ ]:
xgb.score(X_test, y_test)

In [ ]:
%%time
grad=GradientBoostingClassifier(n_estimators=2000,random_state=0,learning_rate=0.1)
grad.fit(X_train, y_train)

In [ ]:
grad.score(X_test, y_test)

In [ ]:
baseline = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100,max_depth=3, min_samples_split=2, min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10)

In [ ]:
%%time
baseline.fit(X_train, y_train)

In [ ]:
baseline.score(X_test, y_test)

In [ ]:
%%time
p_test3 = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 'n_estimators':[100,250,500,750,1000,1250,1500,1750]}

tuning = GridSearchCV(estimator=GradientBoostingClassifier(max_depth=4, min_samples_split=2, min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10), 
            param_grid = p_test3, scoring='accuracy',n_jobs=4,iid=False, cv=5)
tuning.fit(X_train,y_train)

In [ ]:
tuning.cv_results_, tuning.best_params_, tuning.best_score_

In [ ]:
tuning.score(X_test, y_test)

In [ ]:
tuning.best_params_

In [ ]:
tuning.best_score_

In [ ]:
%%time
p_test2 = {'max_depth':[4,41,100,150,300]}
tuning = GridSearchCV(estimator =GradientBoostingClassifier(learning_rate=0.1,n_estimators=1750, min_samples_split=2, min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10), 
            param_grid = p_test2, scoring='accuracy',n_jobs=4,iid=False, cv=5)
tuning.fit(X_train[:40000],y_train[:40000])

In [ ]:
tuning.cv_results_, tuning.best_params_, tuning.best_score_

In [ ]:
tuning.score(X_test, y_test)

In [ ]:
tuning.best_estimator_

In [ ]:
model1 = GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=100,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1750,
              n_iter_no_change=None, presort='auto', random_state=10,
              subsample=1, tol=0.0001, validation_fraction=0.1, verbose=0,
              warm_start=False)
model1.fit(X_train,y_train)
predictors=list(X_train)
feat_imp = pd.Series(model1.feature_importances_, predictors).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Importance of Features')
plt.ylabel('Feature Importance Score')
print('Accuracy of the GBM on test set: {:.3f}'.format(model1.score(X_test, y_test)))
pred=model1.predict(X_test)
print(classification_report(y_test, pred))

In [ ]:
%%time
p_test4 = {'min_samples_split':[2,4,6,13,30,100,150], 'min_samples_leaf':[1,3,5,7]}

tuning = GridSearchCV(estimator =GradientBoostingClassifier(learning_rate=0.1, n_estimators=2000,max_depth=100, subsample=1,max_features='sqrt', random_state=10), 
            param_grid = p_test4, scoring='accuracy',n_jobs=4,iid=False, cv=4)
tuning.fit(X_train[:40000],y_train[:40000])

In [ ]:
%%time
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

In [ ]:
rf.score(X_test, y_test)

In [ ]:
%%time
svc = SVC()
svc.fit(X_train, y_train)

In [ ]:
svc.score(X_test, y_test)